У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [3]:
df = pd.read_csv('customer_segmentation_train.csv')
print(df.columns)

target = 'Segmentation'
X = df.drop(target, axis=1)
y = df[target]

num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

for col in num_cols:
    X[col].fillna(X[col].median(), inplace=True)
for col in cat_cols:
    X[col].fillna(X[col].mode()[0], inplace=True)

X = pd.get_dummies(X, drop_first=True)
y = y.map({'A': 0, 'B': 1, 'C': 2, 'D': 3})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')


<ipython-input-3-e374dcadd78f>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)
<ipython-input-3-e374dcadd78f>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [4]:
df = pd.read_csv('customer_segmentation_train.csv')
target = 'Segmentation'
X_orig = df.drop(target, axis=1)
y_orig = df[target]

for col in X_orig.select_dtypes(include=['int64', 'float64']).columns:
    X_orig[col].fillna(X_orig[col].median(), inplace=True)
for col in X_orig.select_dtypes(include=['object']).columns:
    X_orig[col].fillna(X_orig[col].mode()[0], inplace=True)

X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(X_orig, y_orig, test_size=0.2, random_state=42)

cat_cols = X_train_enc.select_dtypes(include=['object']).columns
for col in cat_cols:
    le = LabelEncoder()
    X_train_enc[col] = le.fit_transform(X_train_enc[col])
    X_test_enc[col] = le.transform(X_test_enc[col])

cat_feature_indices = [X_train_enc.columns.get_loc(col) for col in cat_cols]

orig_numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
X_train_basic = X_train_enc[orig_numeric_cols]

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_basic, y_train_enc)

smote_tomek = SMOTETomek(random_state=42)
X_train_smotetomek, y_train_smotetomek = smote_tomek.fit_resample(X_train_basic, y_train_enc)

smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train_enc, y_train_enc)


<ipython-input-4-c0a7254bd367>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_orig[col].fillna(X_orig[col].median(), inplace=True)
<ipython-input-4-c0a7254bd367>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [5]:
X_test_basic = X_test_enc[orig_numeric_cols]

model_smote = OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42))
model_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = model_smote.predict(X_test_basic)
report_smote = classification_report(y_test_enc, y_pred_smote)
print("SMOTE:")
print(report_smote)

model_smotetomek = OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42))
model_smotetomek.fit(X_train_smotetomek, y_train_smotetomek)
y_pred_smotetomek = model_smotetomek.predict(X_test_basic)
report_smotetomek = classification_report(y_test_enc, y_pred_smotetomek)
print("SMOTE-Tomek:")
print(report_smotetomek)


Classification Report for SMOTE:
              precision    recall  f1-score   support

           A       0.32      0.34      0.33       391
           B       0.22      0.08      0.12       369
           C       0.36      0.40      0.38       380
           D       0.52      0.71      0.60       474

    accuracy                           0.40      1614
   macro avg       0.36      0.38      0.36      1614
weighted avg       0.37      0.40      0.37      1614

Classification Report for SMOTE-Tomek:
              precision    recall  f1-score   support

           A       0.33      0.30      0.31       391
           B       0.16      0.04      0.06       369
           C       0.37      0.50      0.42       380
           D       0.52      0.73      0.61       474

    accuracy                           0.41      1614
   macro avg       0.35      0.39      0.35      1614
weighted avg       0.36      0.41      0.37      1614



**Висновки**

Мною обраною метрикою для порівняння моделей є макро-середній F1-score, адже він рівномірно враховує результати для всіх класів, незалежно від того, скільки представників має кожен з них.

За результатами:

- Модель на базі SMOTE показала макро F1-score 0.36.
- Модель зі SMOTE-Tomek – трохи менше, з макро F1-score 0.35, хоча загальна точність виявилася трохи вищою (41% проти 40%).

Загалом, різниця між моделями є мінімальною.
Напево, дисбаланс у даних не настільки критичний, щоб видалення Tomek links (у SMOTE-Tomek) дало суттєву різницю. АБО Можливо, сам базовий SMOTE вже досить ефективно вирішує проблему нерівномірного розподілу класів, тому додаткове очищення не сильно повпливало.